In [1]:
import os
import wave
import json
import jiwer
import audio

import sys

import whisper

In [2]:
import pandas as pd

In [4]:
# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("/home/ouassim/Downloads/0000.parquet",columns=["br","fr","city"])
# df = pd.read_parquet("/home/wazemour/Stage-Breton/Data/Banque sonore des Dialectes Bretonnes/0000.parquet",columns=["br","fr","city"])

In [5]:
def fix_row(row):
    if 'peurunvan :' in row['fr']:
        # Extract Breton text after "peurunvan :"
        new_text = row['fr'].split('peurunvan :', 1)[1].strip()
        # Replace both Br and Fr columns with this new_text
        row['br'] = new_text
        row['fr'] = new_text
    return row

df = df.apply(fix_row, axis=1)

In [6]:
df

,br,fr,city
0,Pell zo n'eo ket o welet ac'hanon.,Ça fait longtemps qu'il n'est pas venu me voir.,22107
1,N'oc'h ket bet pell zo o welet ac'hanon.,Vous n'êtes pas venus me voir depuis longtemps.,22107
2,Pell zo n'eo ket bet o welet ac'hanon.,Ça fait longtemps qu'il n'est pas venu me voir.,22107
3,Ma... ma zi ne vo ket gwerzhet.,Ma... ma maison ne sera pas vendue.,22107
4,Hon... hon ti ne vo ket gwerzhet.,Notre... notre maison ne sera pas vendue.,22107
...,...,...,...
7286,Ne zeue ket ag ar vro Kemper. Ne zeue ket ag a...,Il ne venait pas du pays de Quimper. Il ne ven...,56247
7287,Bout a oa linad razh d'en-dro ar feunteun. Bou...,Il y avait des orties tout autour de la fontai...,56247
7288,Ne oa ket aes o zennañ.,Ce n'était pas facile de les arracher.,56247
7289,Ne oa ket aes o zennañ.,Ce n'était pas facile de les arracher.,56247


In [7]:
# dataset_path = "/home/wazemour/Stage-Breton/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio"
dataset_path = "/home/ouassim/Desktop/ss/stage/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio"

In [9]:
model = whisper.load_model("small")

In [11]:

from load_ground_truth import load_ground_truth_dict

from transcriber import transcribe_audio

from evaluate_wer_cer import evaluate_wer_cer

In [12]:
def generate_transcription(row):
    file_id = row['file']
    utt_name = f"utt_{file_id:05d}.wav"
    full_path = os.path.join(dataset_path, utt_name)
    return transcribe_audio(full_path, model)

In [25]:
# If your DataFrame is called df
small_df = df.sample(n=200, random_state=42)  # random_state makes it reproducible
small_df

,br,fr,city,file
1057,ur c'harzh,une haie,29032,1057
2654,Mari zo deuet a-benn enno antronoz.,Marie est venue vers eux le lendemain.,29168,2654
4401,"An amzer gwechall, ne oa... ne oa ket, eu... n...","Autrefois, c'était... c'était, euh... ça n'éta...",29230,4401
5843,Piv zo bet o troc'hiñ ar wezenn ?,Qui a coupé l'arbre ?,29293,5843
7177,"Pa oufemp ar wirionez, n'he lavarfemp ket.","Si nous savions la vérité, nous ne la dirions ...",56175,7177
...,...,...,...,...
2925,War an hent am eus gwelet div vaouez.,"Sur le chemin, j'ai vu deux femmes.",29168,2925
2251,Mes pegiz e ra ar re zo e-barzh... e-kostez......,Mes pegiz e ra ar re zo e-barzh... e-kostez......,29150,2251
5777,Ar bedervet deus e verc'hed zo aet da chom da ...,La quatrième de ses filles est allée habiter à...,29293,5777
6815,"Kerzhet a ra ? Eñ... eñ a respont ? O, me a we...",Il fonctionne (ton appareil) ? Il... il répond...,56094,6815


In [26]:
# Step 1: Filter where city starts with '22'
# df_22 = df[df['city'].astype(str).str.startswith('22')].copy()

# Step 2: Add a 'file' column with the original index
small_df['file'] = small_df.index

small_df['transcription'] = small_df.apply(generate_transcription, axis=1)

# Optional: Reset index for a clean new DataFrame
small_df = small_df.reset_index(drop=True)

Transcribing /home/ouassim/Desktop/ss/stage/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio/utt_01057.wav...


/home/ouassim/anaconda3/envs/stt-breton/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:19.200]  ὂ ἕ ˋ başka den ὀ ὢ ᴏ ệ Ử Ổ἟ ᴇ ᴇ ỗ ệ ữ Ồ἟ ᰩ ᴇ ᴨ ᵃ ᴩ, ᴟ �date ᴄ ᴒ ᴜ ᶻ ệ Ổ Ἇ �
Transcribing /home/ouassim/Desktop/ss/stage/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio/utt_02654.wav...


/home/ouassim/anaconda3/envs/stt-breton/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:02.800]  некоторht bread of Lucy physicians is lacked with Daehhhh se if faith is not tied up
Transcribing /home/ouassim/Desktop/ss/stage/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio/utt_04401.wav...


...

[00:00.000 --> 00:02.320]  andin emit j 제품 j miss you TC
[00:02.320 --> 00:03.440]   BRANDON 我们见试一下
Transcribing /home/ouassim/Desktop/ss/stage/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio/utt_06815.wav...


/home/ouassim/anaconda3/envs/stt-breton/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.020 --> 00:00.120]  smaller x standards
[00:01.120 --> 00:01.280]  like here
[00:01.280 --> 00:01.360]  together
[00:01.360 --> 00:05.700]  it doesn't
[00:05.700 --> 00:07.540]  High
Transcribing /home/ouassim/Desktop/ss/stage/Data/Banque sonore des Dialectes Bretonnes/Audio/Audio/utt_06453.wav...


/home/ouassim/anaconda3/envs/stt-breton/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:10.580]  � pergunt Coco ᗋ ᗻᗿball ᗋ ᗵᗾ ᗬᗾ ᗝ ᗘᗿ � mythology ᗒ ᗖᗿ ᗫᗶ э ᕚᗔ� smoked ᕚ ᗗᗽ ᗡ ᗕᗾ, ᗶ ᗊlıᗶ ᓢ � Jug.


In [27]:
small_df

,br,fr,city,file,transcription
0,ur c'harzh,une haie,29032,1057,ὂ ἕ ˋ başka den ὀ ὢ ᴏ ệ Ử Ổ἟ ᴇ ᴇ ỗ ệ ữ Ồ἟ ᰩ ᴇ...
1,Mari zo deuet a-benn enno antronoz.,Marie est venue vers eux le lendemain.,29168,2654,некоторht bread of Lucy physicians is lacked ...
2,"An amzer gwechall, ne oa... ne oa ket, eu... n...","Autrefois, c'était... c'était, euh... ça n'éta...",29230,4401,nd ʻɑm ʻzɔr ʻdwɪ ʃɪl ʻɔa ʻwɑtʃɪt ʻɔa ʻwɑtʃɪt ...
3,Piv zo bet o troc'hiñ ar wezenn ?,Qui a coupé l'arbre ?,29293,5843,PIǜ-ZOO-Bē-ċT bossā advantages.
4,"Pa oufemp ar wirionez, n'he lavarfemp ket.","Si nous savions la vérité, nous ne la dirions ...",56175,7177,ḍ្ ្ ្ៗ ្ ្ម ្៓ ្ ្៓ ្ ្៓ ្៓ ្ ្៓ ្ ្ ្៓
...,...,...,...,...,...
195,War an hent am eus gwelet div vaouez.,"Sur le chemin, j'ai vu deux femmes.",29168,2925,Simpson簽ᗿ � flat
196,Mes pegiz e ra ar re zo e-barzh... e-kostez......,Mes pegiz e ra ar re zo e-barzh... e-kostez......,29150,2251,Korchawe Women went to economised steel andge...
197,Ar bedervet deus e verc'hed zo aet da chom da ...,La quatrième de ses filles est allée habiter à...,29293,5777,andin emit j 제품 j miss you TC BRANDON 我们见试一下
198,"Kerzhet a ra ? Eñ... eñ a respont ? O, me a we...",Il fonctionne (ton appareil) ? Il... il répond...,56094,6815,smaller x standards like here together it doe...


In [28]:
from filter_char import filter_out_chars
from normalizer import normalize_sentence
from utils import pre_process


def process_br_text(br):
    PUNCTUATION = '<>.?!,;:«»“”"()[]/…–—•'
    br = filter_out_chars(br, PUNCTUATION + '*')
    br = normalize_sentence(br, autocorrect=True)
    br = pre_process(br).replace('-', ' ').lower()
    return br

In [29]:
small_df['br_processed'] = small_df['br'].apply(process_br_text)

In [30]:
from jiwer import wer, cer

# Compute WER and CER for each row
small_df['wer'] = small_df.apply(lambda row: wer(row['br_processed'], row['transcription']), axis=1)
small_df['cer'] = small_df.apply(lambda row: cer(row['br_processed'], row['transcription']), axis=1)


In [31]:
small_df = small_df[['br', 'br_processed', 'fr', 'transcription', 'city', 'wer', 'cer', 'file']]
small_df

,br,br_processed,fr,transcription,city,wer,cer,file
0,ur c'harzh,ur c'harzh,une haie,ὂ ἕ ˋ başka den ὀ ὢ ᴏ ệ Ử Ổ἟ ᴇ ᴇ ỗ ệ ữ Ồ἟ ᰩ ᴇ...,29032,16.000000,7.400000,1057
1,Mari zo deuet a-benn enno antronoz.,mari zo deuet a benn enno antronoz,Marie est venue vers eux le lendemain.,некоторht bread of Lucy physicians is lacked ...,29168,2.285714,2.029412,2654
2,"An amzer gwechall, ne oa... ne oa ket, eu... n...",an amzer gwechall ne oa ne oa ket eu ne oa ket...,"Autrefois, c'était... c'était, euh... ça n'éta...",nd ʻɑm ʻzɔr ʻdwɪ ʃɪl ʻɔa ʻwɑtʃɪt ʻɔa ʻwɑtʃɪt ...,29230,1.000000,0.821429,4401
3,Piv zo bet o troc'hiñ ar wezenn ?,piv zo bet o troc'hiñ ar wezenn,Qui a coupé l'arbre ?,PIǜ-ZOO-Bē-ċT bossā advantages.,29293,1.000000,0.935484,5843
4,"Pa oufemp ar wirionez, n'he lavarfemp ket.",pa oufemp ar wirionez n'he lavarfemp ket,"Si nous savions la vérité, nous ne la dirions ...",ḍ្ ្ ្ៗ ្ ្ម ្៓ ្ ្៓ ្ ្៓ ្៓ ្ ្៓ ្ ្ ្៓,56175,2.285714,0.875000,7177
...,...,...,...,...,...,...,...,...
195,War an hent am eus gwelet div vaouez.,war an hent am eus gwelet div vaouez,"Sur le chemin, j'ai vu deux femmes.",Simpson簽ᗿ � flat,29168,1.000000,0.861111,2925
196,Mes pegiz e ra ar re zo e-barzh... e-kostez......,mes pegiz e ra ar re zo e barzh e kostez beg e...,Mes pegiz e ra ar re zo e-barzh... e-kostez......,Korchawe Women went to economised steel andge...,29150,1.000000,0.769565,2251
197,Ar bedervet deus e verc'hed zo aet da chom da ...,ar bedervet deus e verc'hed zo aet da chom da ...,La quatrième de ses filles est allée habiter à...,andin emit j 제품 j miss you TC BRANDON 我们见试一下,29293,1.000000,0.803922,5777
198,"Kerzhet a ra ? Eñ... eñ a respont ? O, me a we...",kerzhet a ra eñ eñ a respont o me a wel oiaou ...,Il fonctionne (ton appareil) ? Il... il répond...,smaller x standards like here together it doe...,56094,1.000000,0.860000,6815


In [33]:
# Assuming your DataFrame has columns: 'city', 'wer', and 'cer'
city_avg = small_df.groupby('city')[['wer', 'cer']].mean()

# Optional: round to 2 decimal places
city_avg = city_avg.round(2)

# Print each city's average WER and CER
for city_code, row in city_avg.iterrows():
    print(f"Average WER and CER for {city_code}:\nWER: {row['wer']}\nCER: {row['cer']}\n")

Average WER and CER for 22107:
WER: 1.17
CER: 0.87

Average WER and CER for 22167:
WER: 2.45
CER: 2.7

Average WER and CER for 22181:
WER: 1.0
CER: 0.86

Average WER and CER for 22230:
WER: 1.39
CER: 1.21

Average WER and CER for 22331:
WER: 1.0
CER: 0.84

Average WER and CER for 22386:
WER: 1.0
CER: 0.91

Average WER and CER for 29032:
WER: 4.81
CER: 2.63

Average WER and CER for 29058:
WER: 0.99
CER: 0.76

Average WER and CER for 29105:
WER: 1.0
CER: 0.71

Average WER and CER for 29122:
WER: 1.0
CER: 0.98

Average WER and CER for 29146:
WER: 1.0
CER: 0.82

Average WER and CER for 29150:
WER: 1.01
CER: 0.87

Average WER and CER for 29153:
WER: 3.09
CER: 3.0

Average WER and CER for 29154:
WER: 1.99
CER: 1.69

Average WER and CER for 29168:
WER: 1.06
CER: 0.93

Average WER and CER for 29180:
WER: 1.12
CER: 0.81

Average WER and CER for 29195:
WER: 1.0
CER: 0.66

Average WER and CER for 29230:
WER: 1.11
CER: 0.89

Average WER and CER for 29236:
WER: 1.33
CER: 0.96

Average WER and CER f

In [34]:
# Filter for cities starting with '22'
small_df_22 = small_df[small_df['city'].str.startswith('22')]
avg_22 = small_df_22[['wer', 'cer']].mean().round(4)


# Filter for cities starting with '22'
small_df_29 = small_df[small_df['city'].str.startswith('29')]
avg_29 = small_df_29[['wer', 'cer']].mean().round(4)


# Filter for cities starting with '56'
small_df_56 = small_df[small_df['city'].str.startswith('56')]
avg_56 = small_df_56[['wer', 'cer']].mean().round(4)



print("Evaluation for --Côtes-d'Armor-- dialects 22xxx :")
print(f"WER: {avg_22['wer']}")
print(f"CER: {avg_22['cer']}\n")

print("Evaluation for --Finistère-- dialects 29xxx :")
print(f"WER: {avg_29['wer']}")
print(f"CER: {avg_29['cer']}\n")

print("Evaluation for --Morbihan-- dialects 56xxx :")
print(f"WER: {avg_56['wer']}")
print(f"CER: {avg_56['cer']}")

Evaluation for --Côtes-d'Armor-- dialects 22xxx :
WER: 1.4713
CER: 1.3781

Evaluation for --Finistère-- dialects 29xxx :
WER: 1.2713
CER: 1.0351

Evaluation for --Morbihan-- dialects 56xxx :
WER: 1.3678
CER: 1.0922


In [35]:
# Assuming 'wer' and 'cer' columns exist
average_wer = small_df['wer'].mean()
average_cer = small_df['cer'].mean()

# Optional: round them
average_wer = round(average_wer, 2)
average_cer = round(average_cer, 2)

print(f"Overall Average WER: {average_wer}")
print(f"Overall Average CER: {average_cer}")

Overall Average WER: 1.31
Overall Average CER: 1.08
